In [1]:
import os, platform, sys
from pprint import pprint

In [2]:
__file__ = os.path.realpath('__file__')
if platform.system() == "Windows":
    from schlichtanders.myos import replace_unc
    __file__ = replace_unc(__file__)
__path__ = os.path.dirname(__file__)
__parent__ = os.path.dirname(__path__)

In [3]:
from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float, Boolean
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker, Session, create_session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base

In [ ]:
from schlichtanders.mycontextmanagers import ignored

In [ ]:
from collections import defaultdict
import csv

In [ ]:
from copy import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
datasetnames = ["boston", "concrete", "energy", "kin8nm", "powerplant", "winered", "yacht"]

In [ ]:
crucial_parameters = ["datasetname", "modelname", "batch_size", "n_normflows", "minus_log_s", "opt_identifier", "opt_momentum",
 "opt_offset", "opt_decay", "opt_step_rate", "best_val_loss"]
def sql_to_dict(sql, keys=crucial_parameters):
    return {k:getattr(sql,k) for k in keys}

In [ ]:
engine = create_engine('sqlite:///' + os.path.join(__path__, "experiment_final_%s1.db" % "boston")) # echo=True
Base = automap_base()
Base.prepare(engine, reflect=True)
Hyper = Base.classes.hyper

In [ ]:
def get_best_ones(datasetname, loglikelihood_lt):
    engine = create_engine('sqlite:///' + os.path.join(__path__, "experiment_final_%s1.db" % datasetname)) # echo=True
    Base = automap_base()
    Base.prepare(engine, reflect=True)
    Hyper = Base.classes.hyper
    session = Session(engine)
    
    best_ones = set()
    for modelname in ["baseline", "radialflow", "planarflow", "mixture",
                "planarflowdet", "radialflowdet"]:
        attr = modelname + "_best_val_loss"
        sql_attr = getattr(Hyper, attr)
        update = session.query(Hyper).order_by(sql_attr.asc()).filter(sql_attr < loglikelihood_lt).all()
        update = update[:2] # take only best 2
        for h in update:
            h.modelname = modelname
            h.best_val_loss = getattr(h, modelname + "_best_val_loss")
        best_ones.update(update)
    return map(sql_to_dict, best_ones)

In [ ]:
from frozendict import frozendict

In [ ]:
best_ones = {}
all_best_ones = []
loglikelihood_lt = 4
for dn in datasetnames:
    best_ones[dn] = get_best_ones(dn, loglikelihood_lt)
    all_best_ones += map(frozendict, best_ones[dn])

In [ ]:
all_unique_best_ones = set(all_best_ones)

In [ ]:
all_unique_best_ones

In [ ]:
with open(os.path.join(__path__, "all_modelname_besttwo_valloss.csv"), "w") as f:
    writer = csv.DictWriter(f, fieldnames=crucial_parameters,
                            quoting=csv.QUOTE_NONNUMERIC)
    writer.writeheader()
    writer.writerows(all_unique_best_ones)